In [1]:
# 메모리내 변수 제거

all = [var for var in globals() if var[0] != "_"]   # globals() 목록의 첫글자가 _ 로 시작하지 않는 자료의 리스트만 가져와서
for var in all:
    del globals()[var]

In [2]:
#%pip install xgboost
#%pip install wordcloud
#%pip install gensim

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import re
import nltk## for language detection

# 박스 출력
import textwrap

# 데이터 수집

In [4]:
train = pd.read_csv('c:/data/project/train.csv')
test = pd.read_csv('c:/data/project/test.csv')

train.head()

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


In [5]:
train['first_party_winner'].value_counts()


first_party_winner
1    1649
0     829
Name: count, dtype: int64

# 데이터 전처리

In [6]:
# 중복 체크 함수인데, 사용하지 않아도 됨. 참고용 셀

def jungbokCheck():
    df = train.drop(columns='ID')
    f_label = df['first_party'].unique() # 한 칼럼의 구성요소들이 서로 중복되지 않게 하기(pandas) # 0619
    s_label = df['second_party'].unique() 

    # 다른 사건임에도 같은 사람들이 중복돼서 등장함
    print('다른 사건인데 중복되는 퍼스트 파티의 갯수 :',len(df['first_party'])-len(f_label))
    print('다른 사건인데 중복되는 세컨드 파티의 갯수 :',len(df['second_party'])-len(s_label))

    # 퍼스트와 세컨드 양쪽 모두 등장하는 사람들
    intersection_label = set(f_label) & set(s_label) # set으로 만들어서 중복되는 지 체크 # 0619
    print('퍼스트와 세컨드에서 모두 등장하는 사람들의 수 : ',len(intersection_label))
    print('퍼스트와 세컨드에서 모두 등장하는 사람들 :',intersection_label)

# jungbokCheck()

In [7]:
# 팩트에 퍼스트파티와 세컨드파티가 포함 되어 있는지 확인하기

df = train.drop(columns='ID')
name_first_nofacts = []
name_second_nofacts = []
name_both_nofacts = []
for i in range(df.shape[0]):
    fact = df['facts'][i]
    first = df['first_party'][i]
    second = df['second_party'][i]
    firstExist = True
    secondExist = True
    countBothNotExist = 0
    splitNum_fir = len(first.split())
    splitNum_sec = len(second.split())
    for e in first.split():
        if e in fact:
            firstExist = True
            break
        else:
            firstExist = False
            splitNum_fir -= 1
        if splitNum_fir <= 0:
            countBothNotExist += 1
            name_first_nofacts.append(i)
    for e in second.split():
        if e in fact:
            secondExist = True
            break
        else:
            secondExist = False
            splitNum_sec -= 1
        if splitNum_sec <= 0:
            countBothNotExist += 1
            name_second_nofacts.append(i)
    if countBothNotExist == 2:
        name_both_nofacts.append(i)

print('사건에서 언급이 안되는 퍼스트 파티의 수 :',len(name_first_nofacts))
print('사건에서 언급이 안되는 세컨드 파티의 수 :',len(name_second_nofacts))
print('사건에서 언급이 안되는 퍼스트와 세컨드의 수 :',len(name_both_nofacts))


사건에서 언급이 안되는 퍼스트 파티의 수 : 405
사건에서 언급이 안되는 세컨드 파티의 수 : 470
사건에서 언급이 안되는 퍼스트와 세컨드의 수 : 82


In [8]:
# 사건에 언급이 안되는 당사자들 빼기

def deleteParty(first=False, second=False, both=True):
    """
        제외시킬 사람들. 양측 제거는 디폴트
    """
    # 양측 모두 제거
    deleted_df = df.copy()
    print('원래 데이터셋 행의 갯수 :',len(df))
    if both == True:
        deleted_df = deleted_df.drop(index=name_both_nofacts)
        print('양측 제거 후 행의 갯수 :',len(deleted_df))
    # 퍼스트 제거
    if first == True:
        left_name_list = set(name_first_nofacts) - set(name_both_nofacts) # 차집합 # 0619
        deleted_df = deleted_df.drop(index=left_name_list)
        print('퍼스트파티 제거 후 갯수 :',len(deleted_df))
    # 세컨드 제거
    if second == True:
        left_name_list = set(name_second_nofacts) - set(name_both_nofacts)
        deleted_df = deleted_df.drop(index=left_name_list)
        print('세컨드파티 후 갯수 :',len(deleted_df))
    
    return deleted_df

name_deleted_df = deleteParty(first=False, second=False, both=True)
name_deleted_df

원래 데이터셋 행의 갯수 : 2478
양측 제거 후 행의 갯수 : 2396


,first_party,second_party,facts,first_party_winner
0,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1
...,...,...,...,...
2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...,1
2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ...",1
2475,Peguero,United States,"In 1992, the District Court sentenced Manuel D...",0
2476,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per...",0


In [9]:
# 승소와 패소 데이터 비율 5:5로 맞추기 # 0619

# 승소와 패소 개수 각각 저장
won_lost_value = name_deleted_df['first_party_winner'].value_counts()
count1 = won_lost_value[1]
count0 = won_lost_value[0]

# 승소한 리스트 인덱스 0부터 다시 지정
reindex_df1 = name_deleted_df[name_deleted_df['first_party_winner']==1]
reindex_df1.reset_index(inplace=True, drop=True)
# 패소한 리스트
reindex_df0 = name_deleted_df[name_deleted_df['first_party_winner']==0]

# 시드 정해놔서 항상 같은 숫자 나오도록 하기
import random
random.seed(0)
remove_list = random.randint(0,count0)

# 패소한 리스트의 갯수만큼 인덱스 랜덤으로 뽑기
lst_1_index = random.sample(range(0,len(reindex_df1)),count0)
lst_1_index.sort()
lst_1_index

# 랜덤인덱스에 맞는 승소 리스트 만들기 (패소리스트 갯수만큼)
newone = reindex_df1.loc[reindex_df1.index.isin(lst_1_index)]

# 승소리스트 패소리스트 합치기
balanced_list = pd.concat([newone, reindex_df0])
balanced_list.reset_index(inplace=True, drop=True)
print('새로운 데이터갯수 :',len(balanced_list))
balanced_list

새로운 데이터갯수 : 1592


,first_party,second_party,facts,first_party_winner
0,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
2,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1
3,"C & A Carbone, Inc., et al.",Town of Clarkstown,"A New York town, Clarkstown, allowed a contrac...",1
4,"David Jennings, et al.","Alejandro Rodriguez, et al.",Sections of the Immigration and Nationality Ac...,1
...,...,...,...,...
1587,Central Laborers' Pension Fund,"Thomas E. Heinz, et al.",Thomas Heinz worked as a construction worker f...,0
1588,United States,Cuauhtemoc Gonzalez-Lopez,"Cuauhtemoc Gonzalez-Lopez hired Joseph Low, an...",0
1589,Peguero,United States,"In 1992, the District Court sentenced Manuel D...",0
1590,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per...",0


In [10]:
df_target = balanced_list[['first_party_winner']]
df_nlp1 = pd.DataFrame(balanced_list.drop(columns='first_party_winner'))
df_nlp1['facts'] = df_nlp1['facts'].str.replace(r'<[^<>]*>', '', regex=True) # 특수 문자 제거
# df_nlp1['facts'] = df_nlp1['facts'].str.replace(r'\d', '', regex=True)  # 숫자 제거
df_nlp1

,first_party,second_party,facts
0,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ..."
1,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...
2,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud..."
3,"C & A Carbone, Inc., et al.",Town of Clarkstown,"A New York town, Clarkstown, allowed a contrac..."
4,"David Jennings, et al.","Alejandro Rodriguez, et al.",Sections of the Immigration and Nationality Ac...
...,...,...,...
1587,Central Laborers' Pension Fund,"Thomas E. Heinz, et al.",Thomas Heinz worked as a construction worker f...
1588,United States,Cuauhtemoc Gonzalez-Lopez,"Cuauhtemoc Gonzalez-Lopez hired Joseph Low, an..."
1589,Peguero,United States,"In 1992, the District Court sentenced Manuel D..."
1590,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per..."


In [11]:
corpus_facts = df_nlp1['facts']
lst_tokens_facts = nltk.tokenize.word_tokenize(corpus_facts.str.cat(sep=" "))

ps = nltk.stem.porter.PorterStemmer()
lem = nltk.stem.wordnet.WordNetLemmatizer()
lst_stopwords = nltk.corpus.stopwords.words("english")

In [12]:
# 문장 깨끗하게 만드는 함수

def utils_preprocess_text(text, flg_stemm=True, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    import re
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    
    ## Tokenize (convert from string to list)
    lst_text = text.split()    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list # 역토큰화, 벡터화 하기 위해서
    text = " ".join(lst_text)
    return text

In [13]:
# 문장 깨끗하게 만드는 함수 사용
df_nlp1["facts_clean"] = df_nlp1["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
df_nlp1["first_clean"] = df_nlp1["first_party"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False))
df_nlp1["second_clean"] = df_nlp1["second_party"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False))

# 갯수가 하나만 있는 단어들 제거
corpus_stopwords = df_nlp1["facts_clean"]
lst_tokens_stopwords = nltk.tokenize.word_tokenize(corpus_stopwords.str.cat(sep=" "))
stop_words=[]

for word, freq in nltk.FreqDist(lst_tokens_stopwords).most_common():
    if freq == 1:
        #print(word)
        stop_words.append(word)

df_nlp1["facts_clean"] = df_nlp1["facts_clean"] .apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False, lst_stopwords=stop_words))

df_nlp1

,first_party,second_party,facts,facts_clean,first_clean,second_clean
0,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",june 27 1962 phil st amant candidate public of...,phil a st amant,herman a thompson
1,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,alabama state court convicted billy joe magwoo...,billy joe magwood,tony patterson warden et al
2,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",april 24 1953 selma alabama intruder broke apa...,william earl fikes,alabama
3,"C & A Carbone, Inc., et al.",Town of Clarkstown,"A New York town, Clarkstown, allowed a contrac...",new york town clarkstown allowed contractor co...,c a carbone inc et al,town of clarkstown
4,"David Jennings, et al.","Alejandro Rodriguez, et al.",Sections of the Immigration and Nationality Ac...,section immigration nationality act require no...,david jennings et al,alejandro rodriguez et al
...,...,...,...,...,...,...
1587,Central Laborers' Pension Fund,"Thomas E. Heinz, et al.",Thomas Heinz worked as a construction worker f...,thomas heinz worked construction worker 20 yea...,central laborers pension fund,thomas e heinz et al
1588,United States,Cuauhtemoc Gonzalez-Lopez,"Cuauhtemoc Gonzalez-Lopez hired Joseph Low, an...",gonzalezlopez hired joseph low attorney repres...,united states,cuauhtemoc gonzalezlopez
1589,Peguero,United States,"In 1992, the District Court sentenced Manuel D...",1992 district court sentenced manuel peguero 2...,peguero,united states
1590,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per...",march 8 1996 st cyr lawful permanent resident ...,immigration and naturalization service,st cyr


In [14]:
# 모델링 전 완전 데이터
df_nlp2 = pd.concat([df_nlp1['facts_clean'],df_nlp1['first_clean'],df_nlp1['second_clean'],df_target['first_party_winner']],axis=1, join='inner')
# df_nlp2 = pd.concat([df_nlp1['facts_clean'],df_nlp1['first_party'],df_nlp1['second_party'],df_target['first_party_winner']],axis=1, join='inner')
df_nlp2

,facts_clean,first_clean,second_clean,first_party_winner
0,june 27 1962 phil st amant candidate public of...,phil a st amant,herman a thompson,1
1,alabama state court convicted billy joe magwoo...,billy joe magwood,tony patterson warden et al,1
2,april 24 1953 selma alabama intruder broke apa...,william earl fikes,alabama,1
3,new york town clarkstown allowed contractor co...,c a carbone inc et al,town of clarkstown,1
4,section immigration nationality act require no...,david jennings et al,alejandro rodriguez et al,1
...,...,...,...,...
1587,thomas heinz worked construction worker 20 yea...,central laborers pension fund,thomas e heinz et al,0
1588,gonzalezlopez hired joseph low attorney repres...,united states,cuauhtemoc gonzalezlopez,0
1589,1992 district court sentenced manuel peguero 2...,peguero,united states,0
1590,march 8 1996 st cyr lawful permanent resident ...,immigration and naturalization service,st cyr,0


In [15]:
#TFIDF / 전체 문서들 중에서 빈도수가 높으면 가중치가 낮고, 특정 문서에서 빈도수가 높으면 가중치를 높게 줌

# vectorizeTF = TfidfVectorizer()
# count_matrix_tf = vectorizeTF.fit_transform(df_nlp2['facts_clean'])
# data_final = count_matrix_tf.toarray()
# data_final = pd.DataFrame(data=data_final, columns=vectorizeTF.get_feature_names_out())
# data_final = pd.concat([data_final,df_nlp2["first_party_winner"]],axis=1,join='inner')
# terms = vectorizeTF.get_feature_names_out()
# data_final

In [16]:
# 카운터 벡터라이즈 / 전체 문서에서 빈도수가 높으면 가중치가 높음
   
# vectorizeCV=CountVectorizer()
# count_matrix = vectorizeCV.fit_transform(df_nlp2['facts_clean'])
# count_array = count_matrix.toarray()
# data_final = pd.DataFrame(data=count_array,columns = vectorizeCV.get_feature_names_out())
# data_final = pd.concat([data_final,df_nlp2["first_party_winner"]],axis=1,join='inner')
# terms = vectorizeCV.get_feature_names_out()
# data_final

In [17]:
def chooseVec(vec,nlp,testSet=False):
    """
        벡터 고르는 함수입니다.
        vec 파라미터로 tf-idf이면 'tf', count-vector는 'cv'.
        nlp는 깨끗하게 전처리된 데이터셋을 보내주세요.
    """
    global terms, vectorize

    if testSet == True: # 테스트 셋일 경우, 새로운 인스턴스를 부여하면 안되니까
        count_matrix = vectorize.transform(nlp['facts_clean'])
    
    if vec == 'tf' and testSet == False:
        #TFIDF / 전체 문서들 중에서 빈도수가 높으면 가중치가 낮고, 특정 문서에서 빈도수가 높으면 가중치를 높게 줌
        vectorize = TfidfVectorizer()
        count_matrix = vectorize.fit_transform(nlp['facts_clean'])

    if vec == 'cv'and testSet == False:
        # 카운터 벡터라이즈 / 전체 문서에서 빈도수가 높으면 가중치가 높음
        vectorize=CountVectorizer()
        count_matrix = vectorize.fit_transform(nlp['facts_clean'])

    count_array = count_matrix.toarray()
    data_final = pd.DataFrame(data=count_array,columns = vectorize.get_feature_names_out())
    data_final = pd.concat([data_final,nlp["first_party_winner"]],axis=1,join='inner')
    terms = vectorize.get_feature_names_out()
    testSet = False
    return data_final
    

In [18]:
data_final = chooseVec('tf', df_nlp2) # td-idf : 'tf' / counter vector : 'cv'

# 데이터 모델링

In [19]:
# X_train, X_test, y_train, y_test = train_test_split(data_final.drop(columns=['first_party_winner']), data_final['first_party_winner'], test_size=0.3,random_state=0)

In [20]:
# from sklearn.decomposition import LatentDirichletAllocation
# lda = LatentDirichletAllocation(n_components=200, random_state=1) # n_component : 토픽 갯수
# pd.DataFrame(lda.fit_transform(X_train))

In [21]:
# 데이터 쪼개기
def splitData(data):
    global X_train, X_test, y_train, y_test
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['first_party_winner']), data['first_party_winner'], test_size=0.3,random_state=0)

# LDA 학습
# def ldaTraining(data, n):
#     """
#         X_trian과 lDA 적용 후 확인할 데이터 n개
#     """
#     # LDA 학습, 단어의 의미구조 파악
#     # 30초정도 걸림
#     global lda, X_train
#     from sklearn.decomposition import LatentDirichletAllocation
#     lda = LatentDirichletAllocation(n_components=200, random_state=1) # n_component : 토픽 갯수
#     lda_data = lda.fit_transform(data)
#     X_train = pd.DataFrame(data=lda_data)
#     # LDA 학습 결과 보기
#     for idx, topic in enumerate(lda.components_):
#         print("Topic %d:" % (idx+1), [(terms[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])


# 모델 만들기
def modeling(params):
    """
        XGB 학습 모델 함수.
        XGB 파라미터를 받습니다.
    """
    global model
    # model = XGBClassifier(**params) # **param_dict
    model = XGBClassifier()
    model.fit(X_train, y_train)
    print(model.get_params())

# 예측하기
def prediction(data):
    """
        테스트 데이터 예측하기
    """
    # lda_data_t = pd.DataFrame(data=lda.transform(data)) # 테스트 lda 적용
    y_pred = model.predict(data)
    y_pred_df = pd.DataFrame(data=y_pred, columns=['data'])
    display(y_pred_df.value_counts())

    # 평가
    from sklearn.metrics import f1_score
    accuracy = accuracy_score(y_test, y_pred) # 정확도
    print("Accuracy : %.2f%%" % (accuracy * 100.0)) 
    print('f1_score :', f1_score(y_test, y_pred)) # 정밀도와 재현율의 조화평균
    print("model accuracy score (X_train, y_train):", model.score(X_test, y_test))

In [22]:
def fullTraining(data):
    # 훈련
    splitData(data)
    # ldaTraining(X_train, 5)

    # XG부스트 모델 적용하기
    default_gs_params =  {'learning_rate': 0.9, 'max_depth': 9, 'subsample': 0.5}
    modeling(default_gs_params)

fullTraining(data_final)

{'objective': 'binary:logistic', 'use_label_encoder': None, 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'gpu_id': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': None, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 100, 'n_jobs': None, 'num_parallel_tree': None, 'predictor': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}


In [23]:
# 테스트 적용
prediction(X_test)

data
1       259
0       219
Name: count, dtype: int64

Accuracy : 51.05%
f1_score : 0.5301204819277109
model accuracy score (X_train, y_train): 0.5104602510460251


In [35]:
model.predict_proba(X_test)

array([[0.6976948 , 0.30230525],
       [0.64265704, 0.357343  ],
       [0.6362013 , 0.36379868],
       ...,
       [0.08203512, 0.9179649 ],
       [0.7457112 , 0.25428876],
       [0.05382466, 0.94617534]], dtype=float32)

In [24]:
# 그리드서치로 최적의 파라미터 값 찾아보기

def findParam():
    from sklearn.model_selection import GridSearchCV
    param_xgb = {
                 "learning_rate":[0.3, 0.6, 0.9], # 0~1 높을 수록 과적합 된다, 만약에 값이 낮으면 n_estimators를 높여야 과적합이 방지
                 "max_depth":[3,5,7,9], # 보통 3~10, 높을 수록 과적합
                 "subsample":[0.5, 0.7, 0.9] # 학습에 사용하는 샘플링 비율 0.5 ~ 1, 높을 수록 과적합
                }    

    gscv_xgb = GridSearchCV (estimator = model, param_grid = param_xgb, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
    gscv_xgb.fit(X_train, y_train)
    gs_params= gscv_xgb.best_params_
    print('XGB 파라미터: ', gs_params)
    print('XGB 예측 정확도: {:.4f}'.format(gscv_xgb.best_score_))

    return gs_params
# 그리드서치 후 파라미터 적용해서 모델링 학습 다시 하기
# modeling(findParam())


### 테스트csv 적용

In [25]:
test

,ID,first_party,second_party,facts
0,TEST_0000,Salerno,United States,The 1984 Bail Reform Act allowed the federal c...
1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",Lexecon Inc. was a defendant in a class action...
2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.","In 2002 and 2003, Fox Television Stations broa..."
3,TEST_0003,Harold Kaufman,United States,During his trial for armed robbery of a federa...
4,TEST_0004,Berger,Hanlon,"In 1993, a magistrate judge issued a warrant a..."
...,...,...,...,...
1235,TEST_1235,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A...",According to Executive Order No. 12807 signed ...
1236,TEST_1236,Whitman,"American Trucking Associations, Inc.",Section 109(a) of the Clean Air Act (CAA) requ...
1237,TEST_1237,Linda A. Matteo and John J. Madigan,William G. Barr,Linda Matteo and John Madigan created a plan f...
1238,TEST_1238,Washington State Apple Advertising Commission,Hunt,"In 1972, the North Carolina Board of Agricultu..."


In [26]:
dfTest = pd.DataFrame(data=test).drop(columns=['ID'])
dfTest['facts'] = dfTest['facts'].str.replace(r'<[^<>]*>', '', regex=True)
dfTest["facts_clean"] = dfTest["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
dfTest["first_clean"] = dfTest["first_party"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False))
dfTest["second_clean"] = dfTest["second_party"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False))

# 갯수가 하나만 있는 단어들 제거
corpus_stopwords = dfTest["facts_clean"]
lst_tokens_stopwords = nltk.tokenize.word_tokenize(corpus_stopwords.str.cat(sep=" "))
stop_words=[]

for word, freq in nltk.FreqDist(lst_tokens_stopwords).most_common():
    if freq == 1:
        #print(word)
        stop_words.append(word)

dfTest["facts_clean"] = dfTest["facts_clean"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False, lst_stopwords=stop_words))

dfTest['first_party_winner'] = np.zeros(len(dfTest)).astype(int)
dfTest = dfTest.drop(columns=['facts','first_party','second_party'])


In [27]:
dfTest

,facts_clean,first_clean,second_clean,first_party_winner
0,1984 bail reform act allowed federal court det...,salerno,united states,0
1,lexecon inc defendant class action lawsuit 28 ...,milberg weiss bershad hynes and lerach,lexecon inc,0
2,2002 2003 fox television station broadcast bil...,no 07582 title federal communications commissi...,fox television stations inc et al,0
3,trial armed robbery federally insured saving l...,harold kaufman,united states,0
4,1993 magistrate judge issued warrant authorizi...,berger,hanlon,0
...,...,...,...,...
1235,according executive order 12807 signed preside...,haitian centers council inc et al,chris sale acting commissioner immigration and...,0
1236,section 109a clean air act caa requires enviro...,whitman,american trucking associations inc,0
1237,linda matteo john madigan created plan utilizi...,linda a matteo and john j madigan,william g barr,0
1238,1972 north carolina board agriculture adopted ...,washington state apple advertising commission,hunt,0


In [28]:
# 벡터 고르기
data_final_test = chooseVec('tf', dfTest, True)
data_final_test

,06,10,100,1000,10000,100000,100to1,101,102,106,...,zita,zivotofsky,zivotofskys,zone,zoneofinterests,zoning,zuni,zurko,zurkos,first_party_winner
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [29]:
# splitData(data_final_test)
def testSplit(data):
    global X_test, y_test
    X_test = data.drop(columns=['first_party_winner'])
    y_test = data['first_party_winner']
testSplit(data_final_test)

In [30]:
def testPredict():
    global predcsv, X_test
    
    # X_test = dataTest.drop(columns=['first_party_winner'])
    # y_test = dataTest['first_party_winner']
    # X_test = pd.DataFrame(data=lda.transform(X_test))
    y_pred_test = model.predict(X_test)
    predcsv = pd.DataFrame(y_pred_test, columns=['first_party_winner'])

testPredict()

In [31]:
def submission():
    submit = pd.read_csv('C:/data/project/sample_submission.csv')
    submit['first_party_winner'] = predcsv
    submit.to_csv('./sample_submission.csv', index=False)
    print('Done')

In [32]:
predcsv.value_counts()

first_party_winner
1                     644
0                     596
Name: count, dtype: int64

In [33]:
submission()

Done


### 셀프 트레이닝

def findGoodSample():
    # 데이터 준비
    proba = model.predict_proba(X_test)
    proba_df = pd.DataFrame(data=proba, columns=['proba_0','proba_1'])

    # 0보다 1의 확률이 높은 데이터 추출
    proba_higher = proba_df[proba_df['proba_1'] > proba_df['proba_0']]
    # 1이 될 확률이 90프로 이상인 자료의 인덱스
    proba_higher_index = proba_higher[proba_higher['proba_1'] > 0.9].index.to_list()

    print('추가되는 샘플 갯수 :',len(proba_higher_index))
    display(proba_higher_index)
    # 인덱스 번호 리턴
    return proba_higher_index


dfTest

def concatSample():
    newIdx = findGoodSample()
    filtered = dfTest.loc[dfTest.index.isin(newIdx)].drop(columns='facts')
    filtered['first_party_winner'] = 1
    dataTest = pd.concat([df_nlp2,filtered]).reset_index(drop=True)
    print('기존 샘플 갯수 :', len(df_nlp2))
    print('새로운 프레임 데이터 갯수 :', len(dataTest))
    return dataTest

data_final_test = chooseVec('tf', concatSample(), True)
fullTraining(data_final_test)
prediction(X_test)

# testPredict(X_test)

# predcsv.value_counts()